**IT 609 - Big Data Processing Lab 4**
- learn about PySpark RDD and DataFrame API and how to use them to
manipulate the data.

Siddharth Kadam **202318015**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

Load Dataset

In [11]:
import requests
import zipfile
import os

url = "http://mattmahoney.net/dc/text8.zip"
response = requests.get(url)
with open("text8.zip", "wb") as f:
    f.write(response.content)

# Unzip the dataset
with zipfile.ZipFile("text8.zip", "r") as zip_ref:
    zip_ref.extractall("data")


For DataFrame

In [13]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create a SparkSession for DataFrame task
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

# Load the text data into a DataFrame
df = spark.read.text("data/text8")

df
spark.stop()

In [16]:
sc.stop()

For RDD

In [17]:
from pyspark import SparkContext

# Initialize SparkContext
sc = SparkContext("local", "RDD Task")

# Load the text data into an RDD
text_rdd = sc.textFile("data/text8")
text_rdd

data/text8 MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

#Excerise

##Task 1
Generate 100 random numbers in range 0 to 10 using numpy randint function with the seed set to 10. Create a RDD using the paral-
lelize function using data generated in previous step. Calculate the fre-
quency of each number (0 - 10) using appropriate function of RDD.

In [20]:
import numpy as np
# generating  100 random numbers in the range 0 to 10
# Set seed for reproducibility
np.random.seed(10)

# Generate 100 random numbers in the range from 0 to 10 using NumPy
random_numbers = np.random.randint(0, 11, size=100)
random_numbers

array([ 9,  4,  0,  1,  9,  0,  1, 10,  8,  9,  0, 10,  8,  6,  4,  3,  0,
        4,  6,  8, 10,  1,  8,  4,  1,  3,  6,  5,  3,  9,  6,  9,  1,  9,
        4,  2,  6,  7,  8, 10,  8,  9,  2,  0,  6,  7,  8,  1,  7,  1,  4,
       10,  0,  8,  5,  4,  7,  8,  8,  2,  6,  2,  8,  8,  6,  6,  5, 10,
        6,  0,  0,  6,  9,  1,  8, 10,  9,  1,  2,  8,  9,  9,  5,  0,  2,
        7,  3,  0,  4,  2,  0,  3,  3,  1,  2,  5,  9,  0, 10,  1])

In [23]:
# Initialize SparkContext
try:
  sc = SparkContext("local", "NumberFreqRDD")
except Exception:
  sc.stop()
  sc = SparkContext("local", "NumberFreqRDD")


# Create an RDD from the generated random numbers
random_numbers_rdd = sc.parallelize(random_numbers)
random_numbers_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [26]:
# Determine each number's frequency by utilizing RDD actions and transformations.
#Each number should be mapped to a key-value pair where the key is the number and the value is 1.
#Next, add up all of the counts for every number by reducing by key.
frequency_counts = random_numbers_rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
for number, frequency in frequency_counts.collect():
    print(f"Number {number}: {frequency} occurrences")

Number 9: 12 occurrences
Number 4: 8 occurrences
Number 0: 12 occurrences
Number 1: 11 occurrences
Number 10: 8 occurrences
Number 8: 14 occurrences
Number 6: 11 occurrences
Number 3: 6 occurrences
Number 5: 5 occurrences
Number 2: 8 occurrences
Number 7: 5 occurrences


##TASK-2:
In this task you will calculate the frequency of each word in text8 dataset mentioned above. Create a RDD using the text8 dataset.

Use appropriate functions of the RDD to get the word frequencies. Fil-
ter the RDD using appropriate function to get the frequencies of words containing the letter ’a’.

In [29]:
try:
  sc = SparkContext("local", "WordFrequencyRDD")
except Exception:
  sc.stop()
  sc = SparkContext("local", "WordFrequencyRDD")

# Load the text8 dataset into an RDD
text_rdd = sc.textFile("data/text8")
# Divide each line into a word and compress the word list.
#Assign a key and value to each word, with the word serving as the key and 1 as the value.
# To add up the counts for every word, reduce by key.
words_rdd = text_rdd.flatMap(lambda line: line.split())
word_count_rdd = words_rdd.map(lambda word: (word, 1))
word_frequency_rdd = word_count_rdd.reduceByKey(lambda a, b: a + b)

# Filter the RDD to get frequencies of words containing the letter 'a'
words_with_a_rdd = word_frequency_rdd.filter(lambda x: 'a' in x[0])

print("Word Frequencies Containing the Letter 'a':")
for word, frequency in words_with_a_rdd.collect():
    print("Word :",word,"Frequency :",frequency)

Streaming output truncated to the last 5000 lines.
Word : variabilily Frequency : 1
Word : internationalised Frequency : 1
Word : beerlao Frequency : 1
Word : chevrotains Frequency : 1
Word : cancore Frequency : 4
Word : levare Frequency : 1
Word : gasena Frequency : 2
Word : sapping Frequency : 2
Word : iguanians Frequency : 1
Word : lagarith Frequency : 1
Word : nevinyrral Frequency : 2
Word : ahims Frequency : 1
Word : archytas Frequency : 1
Word : attp Frequency : 3
Word : wbab Frequency : 1
Word : longisland Frequency : 1
Word : lacoon Frequency : 1
Word : undulate Frequency : 1
Word : hawkwood Frequency : 1
Word : ritterschaften Frequency : 1
Word : stormwarrior Frequency : 1
Word : mascar Frequency : 1
Word : ranchera Frequency : 1
Word : ribao Frequency : 7
Word : zawodowe Frequency : 1
Word : capirola Frequency : 1
Word : nelliady Frequency : 1
Word : carbolith Frequency : 1
Word : emailcash Frequency : 1
Word : batoche Frequency : 8
Word : barrayar Frequency : 4
Word : planes

#DataFrame Task:
Create a Spark dataframe using the iris json data men-
tioned above. Calculate Pearson Correlation between the columns petal-
Length and petalWidth using the appropriate dataframe API. Show the

columns sepalLength, sepalWidth and species for the rows of data that has
petalLength greater than or equal to 1.4 using the appropriate dataframe
API.

In [30]:
!pip install opendatasets
import opendatasets as od

od.download("https://www.kaggle.com/datasets/rtatman/iris-dataset-json-version")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: siddharthkadam1012
Your Kaggle Key: ··········


100%|██████████| 1.31k/1.31k [00:00<00:00, 2.24MB/s]

In [34]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("DataFrame Task") \
    .getOrCreate()

# Read the iris JSON data into a DataFrame
iris_df = spark.read.json("/content/iris-dataset-json-version/iris.json")
iris_df.show()

+---------------+-----------+----------+-----------+----------+-------+
|_corrupt_record|petalLength|petalWidth|sepalLength|sepalWidth|species|
+---------------+-----------+----------+-----------+----------+-------+
|              [|       null|      null|       null|      null|   null|
|           null|        1.4|       0.2|        5.1|       3.5| setosa|
|           null|        1.4|       0.2|        4.9|       3.0| setosa|
|           null|        1.3|       0.2|        4.7|       3.2| setosa|
|           null|        1.5|       0.2|        4.6|       3.1| setosa|
|           null|        1.4|       0.2|        5.0|       3.6| setosa|
|           null|        1.7|       0.4|        5.4|       3.9| setosa|
|           null|        1.4|       0.3|        4.6|       3.4| setosa|
|           null|        1.5|       0.2|        5.0|       3.4| setosa|
|           null|        1.4|       0.2|        4.4|       2.9| setosa|
|           null|        1.5|       0.1|        4.9|       3.1| 

In [35]:
from pyspark.sql.functions import col, corr
# Calculate Pearson correlation between petalLength and petalWidth
correlation = iris_df.select(corr("petalLength", "petalWidth")).collect()[0][0]
print("Pearson Correlation between petalLength and petalWidth:", correlation)

# filtering rows
filtered_df = iris_df.filter(col("petalLength") >= 1.4).select("sepalLength", "sepalWidth", "species")
print("Rows with petalLength >= 1.4:")
filtered_df.show()

Pearson Correlation between petalLength and petalWidth: 0.962865431402796
Rows with petalLength >= 1.4:
+-----------+----------+-------+
|sepalLength|sepalWidth|species|
+-----------+----------+-------+
|        5.1|       3.5| setosa|
|        4.9|       3.0| setosa|
|        4.6|       3.1| setosa|
|        5.0|       3.6| setosa|
|        5.4|       3.9| setosa|
|        4.6|       3.4| setosa|
|        5.0|       3.4| setosa|
|        4.4|       2.9| setosa|
|        4.9|       3.1| setosa|
|        5.4|       3.7| setosa|
|        4.8|       3.4| setosa|
|        4.8|       3.0| setosa|
|        5.7|       4.4| setosa|
|        5.1|       3.5| setosa|
|        5.7|       3.8| setosa|
|        5.1|       3.8| setosa|
|        5.4|       3.4| setosa|
|        5.1|       3.7| setosa|
|        5.1|       3.3| setosa|
|        4.8|       3.4| setosa|
+-----------+----------+-------+
only showing top 20 rows

